In [4]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [8]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
            
    def test(self, data): 
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

In [9]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 4.369318962097168
epoch 10: loss = 2.169266939163208
epoch 20: loss = 0.8587356805801392
epoch 30: loss = 0.48485732078552246
epoch 40: loss = 0.4050009250640869
epoch 50: loss = 0.3527829051017761
epoch 60: loss = 0.37775668501853943
epoch 70: loss = 0.36224353313446045
epoch 80: loss = 0.3371433913707733
epoch 90: loss = 0.3158784508705139
epoch 100: loss = 0.2993887960910797
epoch 110: loss = 0.28548192977905273
epoch 120: loss = 0.273779034614563
epoch 130: loss = 0.2636253237724304
epoch 140: loss = 0.2533580958843231
epoch 150: loss = 0.24338755011558533
epoch 160: loss = 0.23540864884853363
epoch 170: loss = 0.2301664501428604
epoch 180: loss = 0.22702421247959137
epoch 190: loss = 0.22504909336566925
epoch 200: loss = 0.22371362149715424
epoch 210: loss = 0.22278685867786407
epoch 220: loss = 0.22214512526988983
epoch 230: loss = 0.22169719636440277
epoch 240: loss = 0.22139035165309906
Instructions for updating:
Use standard file APIs to check for files with th

array([[6.678357 , 2.7899783, 5.973237 , 2.111731 ]], dtype=float32)